<a href="https://colab.research.google.com/github/davidlealo/100profes/blob/master/modelo_transcripcion_segmentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de Transcripción y Diarización de Audio

Este proyecto combina el poder de **OpenAI Whisper** para la transcripción de audio y **Pyannote** para la diarización (identificación de hablantes) en un archivo de audio. El objetivo principal es tomar un archivo de audio, transcribirlo y asociar cada segmento transcrito con el hablante correspondiente.

## Características

- **Transcripción precisa de audio** utilizando OpenAI Whisper.
- **Diarización automática** para identificar qué segmento pertenece a cada hablante.
- Soporte para múltiples formatos de audio.
- Implementado y probado en Google Colab.

## Tecnologías utilizadas

- **Python**: Lenguaje de programación principal.
- **Whisper (OpenAI)**: Para transcripción de audio.
- **Pyannote**: Para diarización de hablantes.
- **Hugging Face**: Repositorio para modelos como `pyannote/segmentation`.
- **Google Colab**: Entorno para ejecutar y probar el proyecto.

## Configuración del Proyecto

### Prerrequisitos

- Una cuenta en [Hugging Face](https://huggingface.co/) con un token de acceso configurado.
- Una cuenta en OpenAI para utilizar la API de Whisper y obtener una clave API.
- Archivo `.env` con las siguientes variables:
  ```env
  HUGGINGFACE_TOKEN=<tu_huggingface_token>
  OPENAI_API_KEY=<tu_openai_api_key>
  ```

### Instalación

1. Clona este repositorio:
   ```bash
   git clone https://github.com/davidlealo/100profes.git
   cd transcription-diarization
   ```

2. Instala las dependencias:
   ```bash
   pip install -r requirements.txt
   ```

3. Acepta los términos de los modelos en Hugging Face:
   - Ve a [pyannote/segmentation](https://huggingface.co/pyannote/segmentation) y acepta los términos.
   - Ve a [pyannote/speaker-diarization](https://huggingface.co/pyannote/speaker-diarization) y acepta los términos.

4. Configura tu archivo `.env` como se mencionó en los prerrequisitos.

### Uso

1. **Ejecutar en Colab**:
   - Sube el archivo `modelo_transcripcion_segmentacion.ipynb` a Google Colab.
   - Configura las variables de entorno al inicio del notebook.
   - Carga un archivo de audio para procesar.

2. **Prueba con un archivo de audio**:
   - Carga un archivo de audio en el notebook.
   - El pipeline realizará la diarización y transcripción.
   - Los resultados mostrarán qué hablante dijo qué y en qué momento.

### Ejemplo de Salida

Un ejemplo de salida del pipeline sería:

```
Speaker 1 habla desde 0.0 hasta 3.5 segundos: "Hola, ¿cómo estás?"
Speaker 2 habla desde 3.6 hasta 7.2 segundos: "Bien, gracias. ¿Y tú?"
```

## Problemas comunes

- **Error de autenticación en Hugging Face**: Asegúrate de que el token esté configurado correctamente en el archivo `.env`.
- **Modelo no encontrado o no autorizado**: Verifica que has aceptado los términos en la página del modelo.
- **Límite de API excedido**: Si usas la API de OpenAI, verifica tu límite mensual.

## Licencia

Este proyecto está bajo la licencia MIT. Consulta el archivo [LICENSE](LICENSE) para más detalles.

## Contacto

Si tienes preguntas, sugerencias o algún problema con el proyecto, no dudes en contactarme:

- **Email**: [davidlealo@gmail.com](mailto:davidlealo@gmail.com)
- **GitHub**: [https://github.com/davidlealo](https://github.com/davidlealo)
- **LinkedIn**: [https://www.linkedin.com/in/davidlealo/](https://www.linkedin.com/in/davidlealo/)


In [ ]:
# Instalar dependencias necesarias
!pip install openai-whisper pyannote.audio spacy python-dotenv
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 17.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.4 MB/s eta 0:00:00


In [ ]:
# Importar librerías
from pyannote.audio import Pipeline
from whisper import load_model
import spacy
import os
from dotenv import load_dotenv

In [ ]:
# Cargar variables del archivo .env
load_dotenv()
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener el token de Hugging Face
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")

# Iniciar sesión en Hugging Face
login(token=huggingface_token)
print("Autenticación exitosa con Hugging Face.")

Autenticación exitosa con Hugging Face.


In [ ]:
# Probar que funciona token
from huggingface_hub import HfApi

api = HfApi()
user_info = api.whoami(token=HUGGINGFACE_TOKEN)

print(f"Logged in as: {user_info['name']}")


Logged in as: davidlealo


In [ ]:
!pip install --upgrade whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=b7df55259b29e9a6e15e2323544e863f97b0cd5a7f2ec366d52bedf84e006256
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a3508cb14b6
Successfully built whisper


In [ ]:
# Cargar modelos
whisper_model = load_model("base")
nlp = spacy.load("en_core_web_sm")

# Descargar el pipeline de diarización
diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization",
    use_auth_token=HUGGINGFACE_TOKEN
)

# Función para procesar un archivo de audio
def process_audio(file_path):
    # Diarización
    diarization = diarization_pipeline(file_path)
    person_mapping = {}

    for turn, _, speaker in diarization.itertracks(yield_label=True):
        start, end = int(turn.start * 1000), int(turn.end * 1000)
        text = whisper_model.transcribe(file_path, task="transcribe", segment=(start, end))['text']

        # Analizar texto para detectar nombres
        doc = nlp(text)
        names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

        if names:
            person_mapping[speaker] = names[0]

    return {"person_mapping": person_mapping}

# Subir archivo de audio
from google.colab import files
uploaded = files.upload()

# Procesar archivo subido
for file_name in uploaded.keys():
    result = process_audio(file_name)
    print(result)


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 77.9MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

config.yaml:   0%|          | 0.00/500 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/318 [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


...

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

Saving El Dr. House casi comete un error  #shorts.mp3 to El Dr. House casi comete un error  #shorts.mp3


/usr/local/lib/python3.10/dist-packages/torchaudio/_backend/soundfile_backend.py:71: UserWarning: The MPEG_LAYER_III subtype is unknown to TorchAudio. As a result, the bits_per_sample attribute will be set to 0. If you are seeing this warning, please report by opening an issue on github (after checking for existing/closed ones). You may otherwise ignore this warning.
  warnings.warn(


TypeError: DecodingOptions.__init__() got an unexpected keyword argument 'segment'

In [ ]:
import os
import spacy
from whisper import load_model
from pyannote.audio import Pipeline

# Cargar modelos
whisper_model = load_model("base")
nlp = spacy.load("en_core_web_sm")

# Cargar tokens de Hugging Face y OpenAI (si aplica)
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")  # Asegúrate de tener este token en tu .env
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # Sólo necesario si usas servicios de OpenAI

# Descargar el pipeline de diarización
diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization",
    use_auth_token=HUGGINGFACE_TOKEN
)

# Función para procesar un archivo de audio
def process_audio(file_path):
    # Diarización: identificar quién habla y cuándo
    diarization = diarization_pipeline(file_path)
    person_mapping = {}

    # Transcribir todo el audio de una vez
    transcription = whisper_model.transcribe(file_path)
    full_text = transcription["text"]

    # Procesar segmentos de Pyannote para dividir por hablante
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        start, end = int(turn.start * 1000), int(turn.end * 1000)
        segment_text = full_text[start:end]  # Cortar el texto transcrito según los tiempos

        # Analizar texto para detectar nombres
        doc = nlp(segment_text)
        names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

        if names:
            person_mapping[speaker] = names[0]

    return {"person_mapping": person_mapping}

# Subir archivo de audio
from google.colab import files
uploaded = files.upload()

# Procesar archivo subido
for file_name in uploaded.keys():
    result = process_audio(file_name)
    print(result)


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu121. Bad things might happen unless you revert torch to 1.x.


/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

Saving mi-abuelo-no-puede-tomar-refresco-seleneguzmanh-128-ytshorts.savetube.me.mp3 to mi-abuelo-no-puede-tomar-refresco-seleneguzmanh-128-ytshorts.savetube.me.mp3


/usr/local/lib/python3.10/dist-packages/torchaudio/_backend/soundfile_backend.py:71: UserWarning: The MPEG_LAYER_III subtype is unknown to TorchAudio. As a result, the bits_per_sample attribute will be set to 0. If you are seeing this warning, please report by opening an issue on github (after checking for existing/closed ones). You may otherwise ignore this warning.
  warnings.warn(


{'person_mapping': {'SPEAKER_01': 'la tasa'}}


In [ ]:
def process_audio(file_path):
    # Diarización
    diarization = diarization_pipeline(file_path)
    person_mapping = {}

    # Transcribir todo el audio
    transcription = whisper_model.transcribe(file_path)
    full_text = transcription["text"]  # Obtener la transcripción completa del audio

    # Procesar segmentos
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        start, end = int(turn.start * 1000), int(turn.end * 1000)
        segment_text = full_text[start:end]  # Cortar el texto según el rango de tiempo

        # Analizar texto para detectar nombres
        doc = nlp(segment_text)
        names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

        if names:
            person_mapping[speaker] = names[0]

    return {"full_transcription": full_text, "person_mapping": person_mapping}


In [ ]:
# Procesar archivo subido
for file_name in uploaded.keys():
    result = process_audio(file_name)
    print(result)

/usr/local/lib/python3.10/dist-packages/torchaudio/_backend/soundfile_backend.py:71: UserWarning: The MPEG_LAYER_III subtype is unknown to TorchAudio. As a result, the bits_per_sample attribute will be set to 0. If you are seeing this warning, please report by opening an issue on github (after checking for existing/closed ones). You may otherwise ignore this warning.
  warnings.warn(


{'full_transcription': ' El abuelo no puede tomar refresco. Te voy a enseñar un truco. ¿Cuál truco? Solamente cambios la tasa por el vaso para que pienses que esté tomando café. ¿Qué está tomando? Cabecito.', 'person_mapping': {'SPEAKER_01': 'la tasa'}}
